In [1]:
# import dependencies
import pandas as pd
from pathlib import Path
import sqlite3
!pip install pandas psycopg2
!pip install plotly
import plotly.express as px
import psycopg2
from keys import post_username, post_password

Our data has been stored in a cloud server managed by Amazon RDS. The next step is to connect to the database.

In [6]:
# Set up database connection parameters
conn = psycopg2.connect(
    dbname = "team6_project3_db",
    user = post_username,
    password = post_password,
    host = "database-1.croamw4iqxpi.us-east-2.rds.amazonaws.com",
    port = "5432" 
)

query = "SELECT * FROM climate_impact_agriculture;"

In [8]:
# Create the pandas database
third_climate_df = pd.read_sql_query(query, conn)
third_climate_df.head()

C:\Users\nbart.DESKTOP-3OF7M8N\AppData\Local\Temp\ipykernel_32228\500064622.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  third_climate_df = pd.read_sql_query(query, conn)


,year,country,region,crop_type,average_temperature_c,total_precipitation_mm,co2_emissions_mt,crop_yield_mt_per_ha,extreme_weather_events,Irrigation_Access_%,pesticide_use_kg_per_ha,fertilizer_use_kg_per_ha,soil_health_index,adaptation_strategies,economic_impact_million_usd
0,2001,India,West Bengal,Corn,1.55,447.06,15.22,1.737,8,14.54,10.08,14.78,83.25,Water Management,808.13
1,2024,China,North,Corn,3.23,2913.57,29.82,1.737,8,11.05,33.06,23.25,54.02,Crop Rotation,616.22
2,2001,France,Ile-de-France,Wheat,21.11,1301.74,25.75,1.719,5,84.42,27.41,65.53,67.78,Water Management,796.96
3,2001,Canada,Prairies,Coffee,27.85,1154.36,13.91,3.890,5,94.06,14.38,87.58,91.39,No Adaptation,790.32
4,1998,India,Tamil Nadu,Sugarcane,2.19,1627.48,11.81,1.080,9,95.75,44.35,88.08,49.61,Crop Rotation,401.72


Now we've connected to the database and created a pandas DataFrame with the data. Our next step investigate and transform the data. While storing the database and loading the database the column headers have been changed to lowercase. We will change the columns back to their original names to help avoid confusion (some queries have already been done elsewhere using old column names).

In [9]:
# Rename the headers back to their original state, with capitalization
third_climate_df.rename(columns={
    'year': 'Year',
    'country': 'Country',
    'region': 'Region',
    'crop_type': 'Crop_Type',
    'average_temperature_c': 'Average_Temperature_C',
    'total_precipitation_mm': 'Total_Precipitation_mm',
    'co2_emissions_mt': 'CO2_Emissions_MT',
    'crop_yield_mt_per_ha': 'Crop_Yield_MT_per_HA',
    'extreme_weather_events': 'Extreme_Weather_Events',
    'pesticide_use_kg_per_ha': 'Pesticide_Use_KG_per_HA',
    'fertilizer_use_kg_per_ha': 'Fertilizer_Use_KG_per_HA',
    'soil_health_index': 'Soil_Health_Index',
    'adaptation_strategies': 'Adaptation_Strategies',
    'economic_impact_million_usd': 'Economic_Impact_Million_USD'
}, inplace=True)

# Show renamed headers
third_climate_df.head()

,Year,Country,Region,Crop_Type,Average_Temperature_C,Total_Precipitation_mm,CO2_Emissions_MT,Crop_Yield_MT_per_HA,Extreme_Weather_Events,Irrigation_Access_%,Pesticide_Use_KG_per_HA,Fertilizer_Use_KG_per_HA,Soil_Health_Index,Adaptation_Strategies,Economic_Impact_Million_USD
0,2001,India,West Bengal,Corn,1.55,447.06,15.22,1.737,8,14.54,10.08,14.78,83.25,Water Management,808.13
1,2024,China,North,Corn,3.23,2913.57,29.82,1.737,8,11.05,33.06,23.25,54.02,Crop Rotation,616.22
2,2001,France,Ile-de-France,Wheat,21.11,1301.74,25.75,1.719,5,84.42,27.41,65.53,67.78,Water Management,796.96
3,2001,Canada,Prairies,Coffee,27.85,1154.36,13.91,3.890,5,94.06,14.38,87.58,91.39,No Adaptation,790.32
4,1998,India,Tamil Nadu,Sugarcane,2.19,1627.48,11.81,1.080,9,95.75,44.35,88.08,49.61,Crop Rotation,401.72


Now that the DataFrame is adequately transformed we can now investigate it to see what we're dealing with.

In [11]:
# Create variables for unique countries, regions, and their lengths. 
unique_regions = third_climate_df['Region'].unique()
length_unique_regions = len(third_climate_df['Region'].unique())
unique_countries = third_climate_df['Country'].unique()
length_unique_countries = len(third_climate_df['Country'].unique())

# Print results
print(f"This data {length_unique_countries} unique countries, and {length_unique_regions} unique regions")
print(f"These are the unique countries: {unique_countries}")
print(f"These are the unique regions: {unique_regions}")

This data 10 unique countries, and 34 unique regions
These are the unique countries: ['India' 'China' 'France' 'Canada' 'USA' 'Argentina' 'Australia' 'Nigeria'
 'Russia' 'Brazil']
These are the unique regions: ['West Bengal' 'North' 'Ile-de-France' 'Prairies' 'Tamil Nadu' 'Midwest'
 'Northeast' 'New South Wales' 'Punjab' 'North West' 'South East'
 'Grand Est' 'Northwestern' 'Siberian' 'Northwest' 'Victoria'
 'Nouvelle-Aquitaine' 'South' 'Quebec' 'Southeast' 'Ontario' 'East'
 'Pampas' 'Western Australia' 'Volga' 'Maharashtra'
 'Provence-Alpes-Cote d’Azur' 'West' 'Central' 'North Central' 'Patagonia'
 'Queensland' 'South West' 'British Columbia']


Let's find out how many regions are represented for each country in the dataset.

In [12]:
# Group the df by Country and count the number of regions for each
group_df = third_climate_df.groupby('Country')['Region'].nunique().reset_index()
group_df

,Country,Region
0,Argentina,4
1,Australia,4
2,Brazil,4
3,Canada,4
4,China,4
5,France,4
6,India,4
7,Nigeria,4
8,Russia,4
9,USA,4


In [14]:
# Group by country again, but this time show the regions for each instead of counting them.
grouped_df = third_climate_df.groupby("Country")["Region"].unique().reset_index()

# Change to column width of the new df to avoid results being truncated.
pd.set_option('display.max_colwidth', None)
grouped_df

,Country,Region
0,Argentina,"[Northeast, Northwest, Pampas, Patagonia]"
1,Australia,"[New South Wales, Victoria, Western Australia, Queensland]"
2,Brazil,"[North, Northeast, Southeast, South]"
3,Canada,"[Prairies, Quebec, Ontario, British Columbia]"
4,China,"[North, East, South, Central]"
5,France,"[Ile-de-France, Grand Est, Nouvelle-Aquitaine, Provence-Alpes-Cote d’Azur]"
6,India,"[West Bengal, Tamil Nadu, Punjab, Maharashtra]"
7,Nigeria,"[North West, South East, North Central, South West]"
8,Russia,"[Northwestern, Siberian, Volga, Central]"
9,USA,"[Midwest, Northeast, South, West]"


Each country has 4 regions, however there are only 34 unique regions (instead of 40). There is some overlap between regions (e.g. 'South' China and 'South' USA). Now let's find out how many crops are in this dataset.

In [15]:
# Find unique crops
unique_crops = third_climate_df['Crop_Type'].unique()
length_unique_crops = len(unique_crops)

print(f"There are {length_unique_crops} unique crops in this dataset. They are: {unique_crops}")

There are 10 unique crops in this dataset. They are: ['Corn' 'Wheat' 'Coffee' 'Sugarcane' 'Fruits' 'Rice' 'Barley' 'Vegetables'
 'Soybeans' 'Cotton']


We're trying to find a primary key to the data, or at least some more information about what each column represents. It's possible that there is only one crop, per year, per country's region. This would be a unique entry and qualify as a composite key. A new grouped df might help us get a better understanding.

In [41]:
# Break it down by crop type, using size to count the indexes and unstack to reshape the data
crop_type_breakdown = third_climate_df.groupby(['Country', 'Region', 'Year', 'Crop_Type']).size().unstack(fill_value=0)
crop_type_breakdown

Crop_Type                 Barley  Coffee  Corn  Cotton  Fruits  Rice  \
Country   Region    Year                                               
Argentina Northeast 1990       1       1     3       1       2     1   
                    1991       0       2     1       1       2     3   
                    1992       1       0     1       0       2     1   
                    1993       0       1     0       2       0     0   
                    1994       2       1     0       1       1     0   
...                          ...     ...   ...     ...     ...   ...   
USA       West      2020       0       0     2       1       1     0   
                    2021       2       0     1       2       3     1   
                    2022       0       0     0       0       0     2   
                    2023       0       0     2       1       1     2   
                    2024       4       1     1       0       0     1   

Crop_Type                 Soybeans  Sugarcane  Vegetables  Wheat  
Country   Region    Year                                          
Argentina Northeast 1990         0          1           1      0  
                    1991         2          1           2      0  
                    1992         2          0           0      1  
                    1993         0          0           0      1  
                    1994         1          0           1      0  
...                            ...        ...         ...    ...  
USA       West      2020         0          1           0      0  
                    2021         1          1           2      0  
                    2022         0          0           0      0  
                    2023         0          2           1      1  
                    2024         3          1           0      0  

[1398 rows x 10 columns]

This pivot table is heavily trunacted here (it's a large table), however we can see even from this that there are multiple entries for crop per year per region (e.g. Corn appears 3 times in Northeast Argentina in 1990, among other such examples). 
Therefore this dataset does not have a clear primary or composite key. It's not clear exactly what each row represents, aside from perhaps a log entry of harvests. The metadata does not elaborate on this except to say: 

"To assemble this dataset, data was collected from multiple reputable sources, focusing on recent and historical records of temperature, precipitation, and crop yields. The process involved extracting data from agricultural reports, climate monitoring stations, and regional agricultural surveys. Each data point was meticulously reviewed to ensure accuracy and consistency. The dataset was then cleaned and organized to provide a clear and accessible overview of how varying climate conditions influence agricultural productivity across different regions and crop types."